In [3]:
from importlib import reload
import sys
import os
sys.path.insert(0, './../scripts')

In [2]:
import pickle
import numpy as np
import scipy
import torch
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.figure import figaspect
import time

import base_network as base_net
import ring_network as network
import sim_util as su
import ricciardi as ric
import integrate as integ

def imshowbar(fig,ax,A,**kwargs):
    imsh = ax.imshow(A,**kwargs)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = fig.colorbar(imsh, cax=cax, orientation='vertical')
    return cbar

# Build Rosenbaum Network on Ring

In [4]:
ri = ric.Ricciardi()
ri.set_up_nonlinearity('./../scripts/phi_int')

NtE = 50
Nt = NtE*ri.tE
dt = ri.tI/5
T = np.linspace(0,5*Nt,round(5*Nt/dt)+1)
mask_time = T>(4*Nt)

Loading previously saved nonlinearity


In [5]:
jr = np.array([
    [0.04,-0.40],
    [0.12,-0.40]
])
jf = np.array([0.12,0.12])

Kr = 2000
Kf = np.array([1406.25,450])

jmult = 1/224

alfr = 0.25*180*np.ones((2,2))
alff = 0.10*180*np.ones(2)

net = network.RingNetwork(NC=[80,20],NX=30,Nori=500)

net.generate_disorder_two_layer(jr*jmult,alfr,Kr,
                                jf*jmult,alff,Kf)

In [ ]:
start = time.process_time()
sol,_ = integ.sim_dyn(ri,T,0.0,net.M,net.H,net.H,net.C_all[0],net.C_all[1],mult_tau=True,max_min=30)
print('integration time took',time.process_time()-start,'s')